In [1]:
import sqlite3
import pandas as pd
import random

# 1. メモリ上にデータベースを作成（ファイルを作りません）
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# 2. テーブル作成 (Users, Products, Orders)
cursor.execute('''
CREATE TABLE users (
    user_id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER
);
''')

cursor.execute('''
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT,
    category TEXT,
    price INTEGER
);
''')

cursor.execute('''
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    product_id INTEGER,
    order_date TEXT,
    FOREIGN KEY (user_id) REFERENCES users(user_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);
''')

# 3. ダミーデータの投入
# 商品リスト
products = [
    (1, 'MacBook Air', 'Electronics', 120000),
    (2, 'iPhone 15', 'Electronics', 110000),
    (3, 'AirPods Pro', 'Electronics', 30000),
    (4, 'Python Book', 'Books', 3000),
    (5, 'Physics Textbook', 'Books', 5000),
    (6, 'Water', 'Food', 100),
    (7, 'Coffee', 'Food', 500)
]
cursor.executemany('INSERT INTO products VALUES (?,?,?,?)', products)

# ユーザー (A~Eさん)
users = [(i, f'User_{chr(65+i)}', random.randint(20, 40)) for i in range(5)]
cursor.executemany('INSERT INTO users VALUES (?,?,?)', users)

# 注文履歴 (ランダムに生成)
orders = []
order_id = 1
for _ in range(20): # 20回の注文
    u_id = random.randint(0, 4) # User 0-4
    p_id = random.randint(1, 7) # Product 1-7
    orders.append((order_id, u_id, p_id, '2025-01-20'))
    order_id += 1
# 特定の併せ買いパターンを意図的に入れる（MacBookとiPhoneをセットで買うなど）
orders.append((order_id, 0, 1, '2025-01-21')) # AさんがMacBook購入
order_id += 1
orders.append((order_id, 0, 2, '2025-01-21')) # AさんがiPhone購入

cursor.executemany('INSERT INTO orders VALUES (?,?,?,?)', orders)
conn.commit()

print("データベース構築完了！")

# ヘルパー関数: SQLを実行してPandasで見やすく表示
def run_query(query):
    return pd.read_sql_query(query, conn)

データベース構築完了！


In [2]:
print("=== 商品リスト (products) ===")
display(run_query("SELECT * FROM products"))

print("\n=== ユーザーリスト (users) ===")
display(run_query("SELECT * FROM users"))

print("\n=== 注文履歴 (orders) ===")
display(run_query("SELECT * FROM orders LIMIT 10")) # 多いので10件だけ

=== 商品リスト (products) ===


,product_id,product_name,category,price
0,1,MacBook Air,Electronics,120000
1,2,iPhone 15,Electronics,110000
2,3,AirPods Pro,Electronics,30000
3,4,Python Book,Books,3000
4,5,Physics Textbook,Books,5000
5,6,Water,Food,100
6,7,Coffee,Food,500



=== ユーザーリスト (users) ===


,user_id,name,age
0,0,User_A,28
1,1,User_B,39
2,2,User_C,29
3,3,User_D,32
4,4,User_E,37



=== 注文履歴 (orders) ===


,order_id,user_id,product_id,order_date
0,1,1,6,2025-01-20
1,2,4,4,2025-01-20
2,3,1,2,2025-01-20
3,4,3,4,2025-01-20
4,5,1,4,2025-01-20
5,6,4,1,2025-01-20
6,7,3,4,2025-01-20
7,8,1,4,2025-01-20
8,9,1,7,2025-01-20
9,10,4,1,2025-01-20


売上ランキング

In [3]:
sql_1 = '''
SELECT
    p.product_name, COUNT(o.product_id) as sales_count, SUM(p.price) as total_revenue
FROM orders o
JOIN products p ON p.product_id =o.product_id
GROUP BY p.product_id
ORDER BY total_revenue DESC;
'''
run_query(sql_1)

,product_name,sales_count,total_revenue
0,MacBook Air,4,480000
1,iPhone 15,3,330000
2,AirPods Pro,1,30000
3,Python Book,5,15000
4,Physics Textbook,2,10000
5,Coffee,4,2000
6,Water,3,300


ヘビーユーザー特定

In [4]:
sql_2 = '''
SELECT u.name, COUNT(o.order_id) as order_count
FROM orders o
JOIN users u ON u.user_id = o.user_id
GROUP BY u.name
HAVING order_count >= 3;
'''
run_query(sql_2)

,name,order_count
0,User_B,12
1,User_D,3
2,User_E,4


この商品と一緒に買われている商品

In [10]:
sql_3 = '''
SELECT p1.product_name AS Item_A, p2.product_name AS Item_B, COUNT(*) AS pair_count
FROM orders o1
JOIN orders o2 ON o1.user_id = o2.user_id AND o1.order_date = o2.order_date AND o1.product_id < o2.product_id
JOIN products p1 ON o1.product_id = p1.product_id
JOIN products p2 ON o2.product_id = p2.product_id
GROUP BY Item_A, Item_B
ORDER BY pair_count DESC;
'''
run_query(sql_3)

,Item_A,Item_B,pair_count
0,Physics Textbook,Coffee,8
1,Python Book,Coffee,8
2,Water,Coffee,8
3,MacBook Air,Coffee,4
4,MacBook Air,Python Book,4
5,Physics Textbook,Water,4
6,Python Book,Physics Textbook,4
7,Python Book,Water,4
8,iPhone 15,Coffee,4
9,iPhone 15,Python Book,4


データの分析結果、「物理の教科書」を購入するユーザーや「Pythonの本」を購入するユーザーは同時に「コーヒー」を購入する傾向が強く見られた。

これは単なる商品の組み合わせではなく、「勉強のお供にカフェインを摂取する」という具体的な結びつきが存在することを示している。このことから、「学習という集中力を要する行為において、カフェインが不可欠なパートナーである」というユーザーの行動心理を示していると考えられる。

ユーザーが本を買った後、コーヒーを買うためだけに近くのスーパーやコンビニへ行くという「見えない手間」を抱えていること可能性がある。
よって、手間を省く提案として、書籍購入時に「勉強の集中力を高める準備はできていますか？」という文脈でコーヒーを提案することにより、「買いに行く時間を節約してすぐに勉強に集中できる環境」の提供をすることができる。
また、「書籍とコーヒーの定期便」といったサブスクリプションを企画することにより継続的なサポートが可能になると考えられる。